In [1]:
# import functions from utils and porteval
import sys
import os
import pandas as pd

from portopt.portfolio import Portfolio

from portopt.config import load_config
from portopt.utils import write_table

In [ ]:
# required file paths
portfolio_dir = "../data/portfolio"
config_file = os.path.join(portfolio_dir, "config.yml")
holdings_files = os.path.join(portfolio_dir, "holdings")

# to reduce amount of price data that needs to be downloaded, we only use the first holdings file
# comment this out to use all holdings files
num_files = 3
holdings_files = [
    os.path.join(holdings_files, f) 
    for f in os.listdir(holdings_files) 
    if os.path.isfile(os.path.join(holdings_files, f)) and f.endswith('.csv')
][:num_files]

factor_weights_file = os.path.join(portfolio_dir, "asset_class_weights_matrix.csv")

print("portfolio_dir:", portfolio_dir)
print("config_file:", config_file)
print("factor_weights_file:", factor_weights_file)
print("holdings_files:", holdings_files)

In [ ]:
# load config
config = load_config(config_file)
import pprint
pprint.pprint(config)

In [ ]:
# define column formats for write_table function
column_formats = {
    'Ticker': {'width': 14},
    'Level_0': {'width': 14},
    'Level_1': {'width': 14},
    'Level_2': {'width': 14},
    'Level_3': {'width': 14},
    'Level_4': {'width': 14},
    'Level_5': {'width': 14},
    'Level_6': {'width': 14},
    'Factor': {'width': 24},
    'Weight': {'width': 14, 'decimal': 3, 'type':'%'},
    'Account': {'width': 25, 'align': '<'},
    'Name': {'width': 30, 'align': '<'},
    'Short Name': {'width': 20, 'align': '<'},
    'Institution': {'width': 14},
    'Type': {'width': 14},
    'Category': {'width': 14},
    'Family': {'width': 14},
    'Owner': {'width': 14},
    'Quantity': {'width': 10, 'decimal': 3},
    'Original Ticker': {'width': 14},
    'Original Quantity': {'width': 10, 'decimal': 3},
    'Price': {'width': 16, 'decimal': 2, 'prefix': '$'},
    'Total Value': {'width': 16, 'decimal': 2, 'prefix': '$'},
    'Original Value': {'width': 16, 'decimal': 2, 'prefix': '$'},
    'New Value': {'width': 16, 'decimal': 2, 'prefix': '$'},
    'Value Diff': {'width': 16, 'decimal': 2, 'prefix': '$'},
    'Allocation': {'width': 16, 'decimal': 2, 'type':'%'},
    'Original Allocation': {'width': 16, 'decimal': 2, 'type':'%'},
    'New Allocation': {'width': 16, 'decimal': 2, 'type':'%'},
    'Target Allocation': {'width': 16, 'decimal': 2, 'type':'%'},
    'Allocation Diff': {'width': 16, 'decimal': 2, 'type':'%'}
}
column_formats

In [ ]:
# create portfolio object
port = Portfolio(config, factor_weights_file, holdings_files)
port

In [ ]:
write_table(port.getFactors(), column_formats)

In [ ]:
write_table(port.getFactorWeights(), column_formats)

In [ ]:
# get the cononical factor weights matrix
factor_weights_matrix = port.getCanonicalFactorWeightsMatrix(verbose=True)

factor_weights_column_formats = {
    'Factor': {'width': 24}
}
for column in factor_weights_matrix.columns:
    factor_weights_column_formats[column] = {'width': 7, 'decimal': 2, 'type': '%'}

write_table(factor_weights_matrix, factor_weights_column_formats)

In [ ]:
# get variable vectors
variables = port._create_variable_vectors(canonical_matrix=factor_weights_matrix,
                                       account='my_account',
                                       verbose=True)
variables

In [ ]:
# create a target allocation for the portfolio
# Reduce Alt-Crypto, Alt-Gold, and Equity-US by 5 percentage points
results = port.adjust_factor_allocations(
    source_filter={'Level_0': ['Alt', 'Equity'], 'Level_1': ['Crypto', 'Gold', 'US']},
    dest_filter={'Level_0': ['Fixed Income']},
    transfer=0.05,  # 5 percentage points
    verbose=True
)
write_table(results, column_formats)

total_orig_alloc = results['Original Allocation'].sum()
total_new_alloc = results['New Allocation'].sum()
print(f"Total original allocation: {total_orig_alloc:,.2%}")
print(f"Total new allocation: {total_new_alloc:,.2%}")

In [ ]:
# get valid account for testing
# get all available accounts
account_tickers = port.getAccountTickers()
available_accounts = account_tickers.index.get_level_values('Account').unique()

# Print all available accounts
print("Available accounts:")
for account in available_accounts:
    print(f"  - {account}")

# Get the first account name for testing
test_account = available_accounts[0]
print(f"\nUsing account for test: {test_account}")

In [ ]:
# create target allocations vector
# results.loc['Bogus Factor'] = {
#     'Original Allocation': 0.0,
#     'New Allocation': 0.0
# }
# print(results)
target_allocations = port._create_target_factor_allocations_vector(
    results['New Allocation'],
    factor_weights_matrix,
    account=test_account,
    verbose=True
)
target_allocations

In [ ]:
# Now you can use this account to create optimization components
account_optimization_components = port._create_account_optimization_components(
    account=test_account,
    target_factor_allocations=results['New Allocation'],
    verbose=True
)
account_optimization_components

In [14]:
ticker_results, factor_results = port.rebalance(target_allocations,
                                                turnover_penalty=0,
                                                complexity_penalty=0,
                                                min_ticker_alloc=0,
                                                verbose=False)


In [ ]:

write_table(ticker_results, column_formats)
total_alloc = ticker_results['New Allocation'].sum()
print(f"Total allocation: {total_alloc:,.2%}")

print()

write_table(factor_results, column_formats)
total_alloc = factor_results['New Allocation'].sum()
print(f"Total allocation: {total_alloc:,.2%}")